<a id="top"></a>
# Brain Tumor Segmentation Sample Application using TensorFlow* and DNNL*

## Check for latest version
<br><div class=danger><b>Important: Before proceeding, please run the following cell to ensure that you are running the most recent version of this sample.</b></div>

In [ ]:
# Show the current status of this and all documents with ability to update
from qarpo.catalog import DemoCatalog
import os
status = DemoCatalog(os.getcwd(), "Sample").ShowRepositoryControls()

## Prerequisites
This sample requires the following:
- All files are present and in the following directory structure:
    - **argparser.py** - Shared Python* code to perform command line argument parsing
    - **healthcare_no_openvino.ipynb** - This Jupyter* Notebook
    - **healthcare_no_openvino.py** - Python* code for brain tumor segmentation application
    - **images/*.png** - Images used in this Jupyter* Notebook
    - **settings.py** - Shared Python* code containing configuration settings
    - **mkltf.yml** - Dependency information for configuring environment to run the TensorFlow* environment with DNNL* (formerly Intel® MKL-DNN*)libraries
    - **stocktf.yml** - Dependency information for configuring environment to run the standard (stock) TensorFlow* environment
    - **/data/Healthcare_app/data/144x144/Task01_BrainTumour.h5** - Raw MRI input data

It is recommended that you have already read the following from [Get Started on the Intel® DevCloud for the Edge](https://devcloud.intel.com/edge/home/):
- [Overview of the Intel® DevCloud for the Edge](https://devcloud.intel.com/edge/get_started/devcloud/)
- [Overview of the Intel® Distribution of OpenVINO™ toolkit](https://devcloud.intel.com/edge/get_started/openvino/)
- [Getting Started Tutorials](https://devcloud.intel.com/edge/get_started/tutorials/)

<br><div class=note><i><b>Note: </b>It is assumed that the server this sample is being run on is on the Intel® DevCloud for the Edge which has Jupyter* Notebook customizations and all the required libraries already installed.  If you download or copy to a new server, this sample may not run.</i></div>


### Install Keras*
The Python* source code depends upon the ['keras`](https://keras.io/) module which is not present by default and must be installed before running this sample.

Run the following cell to test for and install `keras`.

<br><div class=note><i><b>Note: </b>Installation will take several minutes to complete</i></div>

In [ ]:
try:
    # Test whether keras package is installed
    import keras
    print("keras already installed")
except:
    # Exception while importing keras, try to install it using pip module
    print("Installing keras")
    !python3 -m pip install keras

### Install psutils
The Python* source code depends upon the ['psutils`](https://pypi.org/project/psutil/) module which is not present by default and must be installed before running this sample.

Run the following cell to test for and install `psutils`.

<br><div class=note><i><b>Note: </b>Installation will take several minutes to complete</i></div>

In [ ]:
try:
    # Test whether psutil package is installed
    import psutil
    print("psutil already installed")
except:
    # Exception while importing psutil, try to install it using pip module
    print("Installing psutil")
    !python3 -m pip install psutil

### Create conda environment for TensorFlow* without DNNL* enabled

To run inference using TensorFlow* without DNNL* enabled, we create the [conda](https://docs.conda.io/en/latest/) environment `stocktf` that may be activated before running inference.

Run the next cell to create the `stocktf` conda environment.

In [ ]:
#Check if conda env with stock tensorflow already exists
stocktf_exist = !/data/software/miniconda3/4.7.12/bin/conda info --envs | grep "stocktf "
if not stocktf_exist:
    print("Creating stocktf conda environment...")
    !/data/software/miniconda3/4.7.12/bin/conda config --set auto_activate_base false
    !/data/software/miniconda3/4.7.12/bin/conda env create -f "stocktf.yml"
    print("Done creating stocktf conda environment.")
else:
    print("stocktf conda environment exists")

### Create conda environment for TensorFlow* with DNNL* enabled

To run inference using TensorFlow* with DNNL* enabled, we create the [conda](https://docs.conda.io/en/latest/) environment `dnnltf` that may be activated before running inference.

Run the next cell to create the `dnnltf` conda environment.

In [ ]:
#Check if conda env with optimized tensorflow using mkl already exists
dnnltf_exist = !/data/software/miniconda3/4.7.12/bin/conda info --envs | grep "dnnltf "
if not dnnltf_exist:
    print("Creating dnnltf conda environment...")
    !/data/software/miniconda3/4.7.12/bin/conda config --set auto_activate_base false
    !/data/software/miniconda3/4.7.12/bin/conda env create -f "dnnltf.yml"
    print("Done creating dnnltf conda environment.")
else:
    print("dnnltf conda environment exists")

## Introduction

This sample application serves as a comparison of running inference on a Intel® CPU using TensorFlow* with and without DNNL* enabled to the [same sample application](./healthcare_openvino.ipynb) running inference on a Intel® CPU, GPU, FPGA, and VPU, using the [Intel® Distribution of OpenVINO™ toolkit](https://devcloud.intel.com/edge/get_started/openvino/).  This solution segments brain tumors from raw MRI scans.  The result is a colorized overlay of the original MRI scan highlighting brain tumor matter.

### Key concepts
This sample application includes an example for the following:
- Application:
  - Video and image input is supported using OpenCV
  - OpenCV is used to draw bounding boxes around detected objects, labels, and other information
  - Visualization of the resulting bounding boxes in the output
- Intel® DevCloud for the Edge:
  - Submitting inference as jobs that are performed on different edge compute nodes (rather than on the development node hosting this Jupyter* notebook)
  - Monitoring job status
  - Viewing results and assessing performance for hardware on different compute nodes
- [DNNL* (formerly Intel® MKL-DNN)](https://01.org/dnnl):
  - Run an inference application on multiple hardware devices with and without using DNNL* to accelerate TensorFlow* on CPU


### Application background

To perform brain tumor segmentation (BraTS) the U-Net architecture is used to segment brain tumors from raw MRI scans. With relatively little data a U-Net model can be trained to accurately predict where tumors exist. 

The Dice coefficient (the standard metric for the BraTS dataset used in the study) for our model is about 0.82-0.88.  [Menze et al. reported](http://ieeexplore.ieee.org/document/6975210/) that expert neuroradiologists manually segmented these tumors with a cross-rater Dice score of 0.75-0.85, meaning that the model’s predictions are on par with what expert physicians have made. The below MRI brain scans show highlighted brain tumor matter segmented using a trained model. 

![](images/figure1.png)

### What is U-Net?
Since its introduction two years ago, the [U-Net](https://arxiv.org/pdf/1505.04597) architecture has been used to create deep learning models for segmenting [nerves](https://github.com/jocicmarko/ultrasound-nerve-segmentation) in ultrasound images, [lungs](https://www.kaggle.com/c/data-science-bowl-2017#tutorial) in CT scans, and even [interference](https://github.com/jakeret/tf_unet) in radio telescopes.

The architecture of U-Net is designed like an [auto-encoder](https://en.wikipedia.org/wiki/Autoencoder). It has an encoding path (“contracting”) paired with a decoding path (“expanding”) which gives it the `U` shape.  However, in contrast to the autoencoder, U-Net predicts a pixel-wise segmentation map of the input image rather than classifying the input image as a whole. For each pixel in the original image, it asks the question: “To which class does this pixel belong?” This flexibility allows U-Net to predict different parts of the tumor simultaneously.

![](images/unet.png)

## Brain tumor segmentation application
The brain tumor segmentation application uses the TensorFlow* runtime environment, with and without DNNL*, to perform inference on a raw MRI input image to segment brain tumor matter within the image.  We will setup, run, and view the results for this application for several different hardware devices (CPU. GPU, etc.) available on the compute nodes within the Intel® DevCloud for the Edge.  To accomplish this, we will be performing the following tasks:

1. Install the necessary TensorFlow* runtime environment with and without DNNL* enabled
2. Create the job file used to submit running inference on compute nodes
3. Submit jobs for different compute nodes and monitor the job status until complete
4. View results and assess performance 

### How it works
At startup the brain tumor segmentation application configures itself by parsing the command line arguments.  Once configured, the application and runs inference using the TensorFlow* libraries (with and without DNNL* enabled) on the supplied raw MRI images to segment brain tumors.  The result is a combined image comprising a set of three images: The MRI input, ground truth, and the predicted output from inference with Dice score.  The combined image is also annotated with the amount of time to do inference.

To run the application on the Intel® DevCloud for the Edge, a job is submitted to an edge compute node with a Intel® CPU. After inference on the input is completed, the output is stored in the appropriate `results/<architecture>/` directory.  The results are then viewed within this Jupyter* Notebook using the `outputHTML` image display utility.

The application and inference code for this sample is already implemented in the two Python* files: [`healthcare_no_openvino.py`](./healthcare_openvino.py) and supporting [`argparser.py`](./argparser.py) and [`settings.py`](./settings.py).

The following sections will guide you through configuring and running the brain tumor segmentation application.

### Configuration
The following sections describe all the necessary configuration to run the brain tumor segmentation application.
#### Command line arguments
The application is run from the command line using the following format:
```bash
python3 healthcare_no_openvino.py <arguments...>
```
The required command line _<arguments...>_ to run the Python* executable [`healthcare_no_openvino.py`](./healthcare_no_openvino.py) are:
- **-r** - The path to where the output video file will be stored

### Imports
We begin by importing all the Python* modules that will be used within this Jupyter* Notebook to run and display the results of the brain tumor segmentation application on the Intel® DevCloud for the Edge:
- [os](https://docs.python.org/3/library/os.html#module-os) - Operating system specific module (used for file name parsing)
- [time](https://docs.python.org/3/library/time.html#module-time) - Time tracking module (used for measuring execution time)
- [matplotlib.pyplot](https://matplotlib.org/) - pyplot is used for displaying output images
- [sys](https://docs.python.org/3/library/sys.html#module-sys) - System specific parameters and functions
- [qarpo.demoutils](https://github.com/ColfaxResearch/qarpo) - Provides utilities for displaying results and managing jobs from within this Jupyter* Notebook

Run the following cell to import the Python* dependencies needed.

<br><div class=tip><b>Tip: </b>Select a cell and then use **Ctrl+Enter** to run that cell.</div>

In [ ]:
import matplotlib.pyplot as plt
import os
import time
import sys
from qarpo.demoutils import *
print('Imported Python modules successfully.')

### Run inference
The following sections will go through the steps to run our inference application on the Intel® DevCloud for the Edge. 

#### Create the job file
We will run inference on several different edge compute nodes present in the Intel® DevCloud for the Edge. We will send work to the edge compute nodes by submitting the corresponding non-interactive jobs into a queue. For each job, we will specify the type of the edge compute server that must be allocated for the job.

The job file is a [Bash](https://www.gnu.org/software/bash/) script that serves as a wrapper around the Python* executable of our application that will be executed directly on the edge compute node.  One purpose of the job file is to simplify running an application on different compute nodes by accepting a few arguments and then performing accordingly any necessary steps before and after running the application executable.  

For this sample, the job file we will be using is already written for you and appears in the next cell.  The job file will be submitted as if it were run from the command line using the following format:
```bash
healthcare_no_openvino_job.sh <output_directory> <dnnl_enabled>
```
Where the job file input arguments are:
- <*output_directory*> - Output directory to use to store output files
- <*dnnl_enabled*> - Whether to enable (`1`) or disable (`0`) using the DNNL libraries

Based on the input arguments, the job file will do the following:
- Change to the working directory `PBS_O_WORKDIR` where this Jupyter* Notebook and other files appear on the compute node
- Create the <*output_directory*>
- Activate the appropriate conda environment 
- Run the application Python* executable with the appropriate command line arguments

Run the following cell to create the `healthcare_no_openvino_job.sh` job file.  The [`%%writefile`](https://ipython.readthedocs.io/en/stable/interactive/magics.html#cellmagic-writefile) line at the top will write the cell contents to the specified job file `healthcare_no_openvino_job.sh`.

In [ ]:
%%writefile healthcare_no_openvino_job.sh

# Store input arguments: <output_directory> <dnnl_enabled> 
OUTPUT_FILE=$1
DNNL_ENABLED=$2

# The default path for the job is the user's home directory,
#  change directory to where the files are.
cd $PBS_O_WORKDIR

# Make sure that the output directory exists.
mkdir -p $OUTPUT_FILE

# Activate the necessary conda environment
if [ "$DNNL_ENABLED" = "1" ]; then
    CONDA_ENV="dnnltf"
else
    CONDA_ENV="stocktf"
fi
echo "Activating the $CONDA_ENV conda environment"
conda activate $CONDA_ENV

# Run the brain tumor segmentation code
python3 healthcare_no_openvino.py -r $OUTPUT_FILE


#### How to submit a job

Now that we have the job script, we can submit jobs to edge compute nodes in the Intel® DevCloud for the Edge.  To submit a job, the `qsub` command is used with the following format:
```bash
qsub <job_file> -N <JobName> -l <nodes> -F "<job_file_arguments>" 
```
We can submit TODO.sh to several different types of edge compute nodes simultaneously or just one node at a time.

There are three options of `qsub` command that we use for this:
- <*job_file*> - This is the job file we created in the previous step
- `-N` <*JobName*> : Sets name specific to the job so that it is easier to distinguish  between it and other jobs
- `-l` <*nodes*> - Specifies the number and the type of nodes using the format *nodes*=<*node_count*>:<*property*>[:<*property*>...]
- `-F` "<*job_file_arguments*>" - String containing the input arguments described in the previous step to use when running the job file

*(Optional)*: To see the available types of nodes on the Intel® DevCloud for the Edge, run the following cell.

In [ ]:
!pbsnodes | grep compnode | awk '{print $3}' | sort | uniq -c

In the above output from executing the previous cell, the properties describe the node, and the number on the left is the number of available nodes of that architecture.

#### Submit jobs

Each of the cells in the subsections below will submit a job to be run on different edge compute nodes. The output of each cell is the _JobID_ for the submitted job.  The _JobID_ can be used to track the status of the job.  After submission, a job will go into a waiting queue before running once the requested compute nodes become available.

<br><div class=note><i><b>Note: </b>You may submit all jobs at once or one at a time.</i></div> 

<br><div class=tip><b>Tip: </b>**Shift+Enter** will run the cell and automatically move you to the next cell. This allows you to use **Shift+Enter** multiple times to quickly run through multiple cells, including markdown cells.</div>

##### Submit to an edge compute node with an Intel® CPU without DNNL* enabled
In the cell below, we submit a job to an edge node with an [Intel® Core™ i5-6500TE](https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz.html) processor. The inference workload will run on the CPU.

In [ ]:
#Submit job to the queue
job_id_core_stocktf = !qsub healthcare_no_openvino_job.sh -l nodes=1:i5-6500te -F "results/core_stocktf/ 0" -N braints_core_stocktf
print(job_id_core_stocktf[0])
#Progress indicators
if job_id_core_stocktf:
    progressIndicator('results/core_stocktf', 'i_progress_'+job_id_core_stocktf[0]+'.txt', "Inference", 0, 100)

##### Submit to an edge compute node with Intel® Xeon® CPU without DNNL* enabled
In the cell below, we submit a job to an edge node with an [Intel® Xeon® Processor E3-1268L v5](https://ark.intel.com/products/88178/Intel-Xeon-Processor-E3-1268L-v5-8M-Cache-2-40-GHz.html). The inference workload will run on the CPU.

In [ ]:
#Submit job to the queue
job_id_xeon_stocktf = !qsub healthcare_no_openvino_job.sh  -l nodes=1:e3-1268l-v5 -F "results/xeon_stocktf/ 0" -N braints_xeon_stocktf
print(job_id_xeon_stocktf[0])
#Progress indicators
if job_id_xeon_stocktf:
    progressIndicator('results/xeon_stocktf', 'i_progress_'+job_id_xeon_stocktf[0]+'.txt', "Inference", 0, 100)

##### Submit to an edge compute node with an Intel® CPU with DNNL* enabled
In the cell below, we submit a job to an edge node with an [Intel® Core™ i5-6500TE](https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz.html) processor. The inference workload will run on the CPU.

In [ ]:
#Submit job to the queue
job_id_core_dnnltf = !qsub healthcare_no_openvino_job.sh -l nodes=1:i5-6500te -F "results/core_dnnltf/ 1" -N braints_core_dnnltf
print(job_id_core_dnnltf[0])
#Progress indicators
if job_id_core_dnnltf:
    progressIndicator('results/core_dnnltf', 'i_progress_'+job_id_core_dnnltf[0]+'.txt', "Inference", 0, 100)

##### Submit to an edge compute node with Intel® Xeon® CPU with DNNL* enabled
In the cell below, we submit a job to an edge node with an [Intel® Xeon® Processor E3-1268L v5](https://ark.intel.com/products/88178/Intel-Xeon-Processor-E3-1268L-v5-8M-Cache-2-40-GHz.html). The inference workload will run on the CPU.

In [ ]:
#Submit job to the queue
job_id_xeon_dnnltf = !qsub healthcare_no_openvino_job.sh  -l nodes=1:e3-1268l-v5 -F "results/xeon_stocktf/ 1" -N braints_xeon_dnnltf
print(job_id_xeon_dnnltf[0])
#Progress indicators
if job_id_xeon_dnnltf:
    progressIndicator('results/xeon_dnnltf', 'i_progress_'+job_id_xeon_dnnltf[0]+'.txt', "Inference", 0, 100)

### Monitor job status

To check the status of the jobs that have been submitted, use the `qstat` command.  The custom Jupyter* Notebook widget `liveQstat()` is provided to display the output of `qstat` with live updates.  

Run the following cell to display the current job status with periodic updates. 

In [ ]:
liveQstat()

You should see the jobs that you have submitted (referenced by the `JobID` that gets displayed right after you submit the jobs in the previous step).
There should also be an extra job in the queue named `jupyterhub-singleuser`: this job is your current Jupyter* Notebook session which is always running.

The `S` column shows the current status of each job: 
- If the status is `Q`, then the job is queued and waiting for available resources
- If ste status is `R`, then the job is running
- If the job is no longer listed, then the job has completed

<br><div class=note><i><b>
Note: The amount of time spent in the queue depends on the number of users accessing the requested compute nodes. Once the jobs for this sample application begin to run, they should take from 1 to 5 minutes each to complete.
</b></i></div>

<br><div class=danger><b>Wait!: </b>Please wait for the inference jobs and video rendering to complete before proceeding to the next step to view results.</div>

### View results

Once the jobs have completed, the queue system outputs the stdout and stderr streams of each job into files with names of the forms <*JobName*>.o<*JobID*> and <*JobName*>.e<*JobID*>, respecitvely.  The *JobName* corresponds to the `-N` option when submitting the job using the `qsub` command.  

The output image files for each job are written to the files `pred*.png` located in the directory `results/<device>` that was specified as the output directory to the job file.  We will now use the `outputHTML()` utility to display the output image files within this Jupyter* Notebook.  Calling `outputHTML()` from a Python* cell follows the form:
```python
outputHTML(title, directory, extension)
```
The parameters are:
- *title* - Title to put at the top of the displayed output
- *directory* - Directory containing output image files to display
- *extension* - File extension of output image files to display

Run the cells below to display the output images.

<br><div class=note><i><b>Note: </b>See [`demoutils.py`](../../../demoTools/demoutils.py) for more information on how the results are displayed in Jupyter* Notebooks.</i></div>

##### View results from an Intel® CPU without DNNL* enabled

In [ ]:
outputHTML('Intel Core CPU', 
          'results/core_stocktf', '.png')

##### View results from an Intel® Xeon® CPU without DNNL* enabled

In [ ]:
videoHTML('Intel Xeon CPU',
          'results/xeon_stocktf', '.png')

##### View results from an Intel® CPU with DNNL* enabled

In [ ]:
outputHTML('Intel Core CPU', 
          'results/core_dnnltf', '.png')

##### View results from an Intel® Xeon® CPU with DNNL* enabled

In [ ]:
videoHTML('Intel Xeon CPU',
          'results/xeon_dnnltf', '.png')

### View and assess performance results

The running time of each inference task is recorded in `results/<device>/stats.txt`. Run the cell below to plot the results of all jobs side-by-side. Lower values mean better performance for **Inference Engine Processing Time** and higher values mean better performance for **Inference Engine FPS**. When comparing results, please keep in mind that some architectures are optimized for highest performance, others for low power or other metrics.

In [ ]:
# Create table of <architecture>, <title> for plotting
arch_list = [('core_stocktf', 'Intel Core\ni5-6500TE\nCPU without DNNL'),
             ('xeon_stocktf', 'Intel Xeon\nE3-1268L v5\nCPU without DNNL'),
             ('core_dnnltf', 'Intel Core\ni5-6500TE\nCPU with DNNL'),
             ('xeon_dnnltf', 'Intel Xeon\nE3-1268L v5\nCPU with DNNL')]
# For each archtecture in table, create path to stats file or placeholder 
stats_list = []
for arch, a_name in arch_list:
    # if job_id_<architecture> exists, the job was run and has a stats file
    if 'job_id_'+arch in vars():
        stats_list.append(('results/{arch}/stats.txt'.format(arch=arch), a_name))
    else:
        stats_list.append(('placeholder'+arch, a_name))
# Plot the execution time from the stats files
summaryPlot(stats_list, 'Architecture', 'Time, seconds', 'Inference Engine Processing Time', 'time')
# Plot the frames per second from the stats files
summaryPlot(stats_list, 'Architecture', 'Frames per second', 'Inference Engine FPS', 'fps')

## Next steps
- [More Jupyter* Notebook Samples](https://devcloud.intel.com/edge/advanced/sample_applications/) - additional sample applications 
- [Jupyter* Notebook Tutorials](https://devcloud.intel.com/edge/get_started/tutorials) - sample application Jupyter* Notebook tutorials
- [Intel® Distribution of OpenVINO™ toolkit Main Page](https://software.intel.com/openvino-toolkit) - learn more about the tools and use of the Intel® Distribution of OpenVINO™ toolkit for implementing inference on the edge


## About this notebook

For technical support, please see the [Intel® DevCloud Forums](https://software.intel.com/en-us/forums/intel-devcloud-for-edge)

<p style=background-color:#0071C5;color:white;padding:0.5em;display:table-cell;width:100pc;vertical-align:middle>
<img style=float:right src="https://devcloud.intel.com/edge/static/images/svg/IDZ_logo.svg" alt="Intel DevCloud logo" width="150px"/>
<a style=color:white>Intel® DevCloud for the Edge</a><br>   
<a style=color:white href="#top">Top of Page</a> | 
<a style=color:white href="https://devcloud.intel.com/edge/static/docs/terms/Intel-DevCloud-for-the-Edge-Usage-Agreement.pdf">Usage Agreement (Intel)</a> | 
<a style=color:white href="https://devcloud.intel.com/edge/static/docs/terms/Colfax_Cloud_Service_Terms_v1.3.pdf">Service Terms (Colfax)</a>
</p>